# Statistics

## imports

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.autolayout"] = True
plt.rcParams["savefig.facecolor"] = (0.0, 0.0, 0.0, 0)

In [ ]:
import numpy as np

In [ ]:
from numpy import sqrt

In [ ]:
np.__version__

In [ ]:
#from scipy.stats import poisson, t, norm

In [ ]:
import scipy.stats 

In [ ]:
import scipy

In [ ]:
import math

## random numbers

In [ ]:
rng = np.random.default_rng(seed=20071973)

In [ ]:
N = 1000000

tiles = 100000

mu = N/tiles

In [ ]:
f'N = {N:1.0e}, t = {tiles:1.0e}, mu = {mu:2.0f}'

In [ ]:
mu

In [ ]:
## draw random numbers, which are the tile indices
random_numbers = rng.integers(0, tiles, size = N)

## count the hits in each tile
counts_in_tiles = np.unique_counts(random_numbers).counts

## some tiles were never hit, [tiles - len(counts_in_tiles)] have zero counts
## extend the array with this number of zeros
counts_in_tiles = np.append(counts_in_tiles, np.zeros(tiles - len(counts_in_tiles), dtype = counts_in_tiles.dtype))

In [ ]:
len(counts_in_tiles)

In [ ]:
rng.shuffle(counts_in_tiles)

In [ ]:
## make trhe histogram of counts

counts = np.bincount(counts_in_tiles)

In [ ]:
np.sum(counts) == tiles

In [ ]:
values = np.arange(len(counts))

In [ ]:
plt.stairs(counts, np.append(values, values[-1]+1) - 0.5,
           fill=True
          )

plt.yscale('log')

plt.savefig('DropDistribution.svg')

## study the distribution

### mean
The mean of the list of values is
\begin{equation}
\bar{x} = \frac{1}{n} \sum_{i=1}^n x_i
\end{equation}


In [ ]:
np.sum(counts_in_tiles)/len(counts_in_tiles)

In [ ]:
np.mean(counts_in_tiles)

The expected value of a distribution is
\begin{equation}
\mu = \frac{1}{\sum_{i = 1}^{n} w_i} \sum_{i = 1}^{n} w_i x_i = \sum_{i = 1}^{n} p_i x_i
\end{equation}
with $w_i$ being the weigths of the individual values or $p_i$ being the $p_i$ being the probabilities such that $\sum_{i = 1}^{n} p_i = 1$.

In [ ]:
counts

In [ ]:
mu = np.sum(values*counts)/np.sum(counts)

In [ ]:
mu

### variance
The variance of the list of equally likely values is
\begin{equation}
\sigma^2 = \frac{1}{n} \sum_{i=1}^n \left( x_i - \bar{x} \right)^2
\end{equation}
with $\sigma$ being the standard deviation.

In [ ]:
np.sum((counts_in_tiles - np.mean(counts_in_tiles))**2)/len(counts_in_tiles)

In [ ]:
np.var(counts_in_tiles)

Similarily, the variance of a distribution is
\begin{equation}
\sigma^2 = \frac{1}{\sum_{i = 1}^{n} w_i} \sum_{i = 1}^{n} w_i (x_i - \mu)^2 = \sum_{i = 1}^{n} p_i (x_i - \mu)^2
\end{equation}
with $\mu$ being the expected value (or mean) of the distribution,  $w_i$ being the weigths of the individual values or $p_i$ being the $p_i$ being the probabilities such that $\sum_{i = 1}^{n} p_i = 1$.

In [ ]:
np.sum((values-mu)**2*counts)/np.sum(counts)

### skewness

We can also calculate the third moment of the distribution
\begin{equation}
\mu_3 = \sum_{i=1}^n p_i \left( x_i - \mu \right)^3
\end{equation}
with $p_i$ being the probabilities. For the list of numbers $p_i = 1/n$. This is related to the skewness and is a measure of the asymmetry of the distribution.

In [ ]:
mu_3 = np.sum((counts_in_tiles - np.mean(counts_in_tiles))**3)/len(counts_in_tiles)

In [ ]:
mu_3

There are different normalisations that can be used, '''scipy.stats''' implements the Fisher-Pearson coefficient of skewness,
\begin{equation}
g_1 = \frac{\mu_3}{\sigma^3}
\end{equation}

In [ ]:
mu_3/np.var(counts_in_tiles)**(3/2)

In [ ]:
from scipy.stats import skew

In [ ]:
skew(counts_in_tiles)

The details will not be discussed here. The main point is to show that the distribution is not symmetric, so it cannot be a normal distribution!

### standard deviation



In [ ]:
mu = np.mean(counts_in_tiles)
var = np.var(counts_in_tiles)
sigma = np.std(counts_in_tiles)

In [ ]:
mu, var, sqrt(var), sigma

In [ ]:
dev = (counts_in_tiles - mu) / sigma

In [ ]:
plt.hist(dev, bins = np.arange(int(np.min(dev)), int(np.max(dev))+2))

plt.xlabel('$(x - \\mu)/\\sigma$')

#plt.yscale('log')

plt.savefig('Deviation.svg')

In [ ]:
np.sum(np.abs(dev) < 3)/len(counts_in_tiles)

99.7% within $3\sigma$ is what we expect for a normal distribution.

In [ ]:
1 - 2*scipy.stats.norm.sf(3)

In [ ]:
np.sum(np.abs(dev) < 1)/len(counts_in_tiles)

74% within $1\sigma$ is higher than expected. But the distribution is not normal!

In [ ]:
1 - 2*scipy.stats.norm.sf(1)

## Estimating the Rate

Now we will try to estimate the true value $\mu$

In [ ]:
mu

from our measurements. We are looking for an uncertainty with a confidence level of

In [ ]:
confidence_level = 1 - 2*scipy.stats.norm.sf(1)

In [ ]:
confidence_level

### single measurement

We can use the counts in each tile as a measurement for the overall rate. Let's take one standard deviation as the square root of the measurement.

In [ ]:
dev = (counts_in_tiles - mu)/sqrt(counts_in_tiles)

In [ ]:
dev

In [ ]:
np.any(np.isinf(dev))

We see already that this fails in the cases where we have zero counts.

In [ ]:
plt.hist(dev, 
         bins = np.arange(int(np.min(dev[~np.isinf(dev)])), int(np.max(dev[~np.isinf(dev)]))+2)
        )

plt.xlabel('$(x - \\overline{x}) / \\sigma$')

plt.savefig('Deviation_SingleEstimate.svg')

In [ ]:
np.sum(np.abs(dev) < 1)/len(counts_in_tiles)

In [ ]:
np.sum(np.abs(dev) < 3)/len(counts_in_tiles)

This seems to be a bad estimate. In particular for the rare case of zero counts we do not even have an estimate of the uncertainty.

In [ ]:
plt.hist(sqrt(counts_in_tiles))

In [ ]:
np.mean(sqrt(counts_in_tiles))

We will be able to do better than that.

In [ ]:
scipy.stats.t.isf((1-confidence_level)/2, 1)

### average over several measurements
Let's make the measurement several times, i.e. we are counting several tiles.

In [ ]:
sample_size = 10

In [ ]:
samples = np.reshape(counts_in_tiles, shape = (-1, sample_size))

In [ ]:
samples.shape

In [ ]:
len(samples)

In [ ]:
sample_means = np.mean(samples, axis = (1))

In [ ]:
sample_means

In [ ]:
plt.hist(sample_means, bins = np.arange(int(np.min(sample_means)), int(np.max(sample_means))+2) -0.5)

plt.xlabel(f'mean of {sample_size} measurements')

plt.savefig('MeanOf10.svg')

The standard deviation of the estimate of the mean is
\begin{equation}
\sigma_\text{mean} = t_{\alpha, n - 1} \frac{\sigma_\text{sample}}{\sqrt{n}}
\end{equation}
where
\begin{equation}
\sigma_\text{sample} = \sqrt{\frac{1}{n - 1} \sum_{i=1}^n \left( x_i - \bar{x} \right)^2}
\end{equation}
is the sample's standard deviation and $n$ is the number of measurements used for the mean.
The $n - 1$ compared to the distribution's standard deviation is the Bessel correction.

$t_{\alpha, n - 1}$ is the percentile of the Student's $t$ distribution. For a given confidence level $cf$ it is such that
\begin{equation}
\int_{-t_{\alpha, n - 1}}^{t_{\alpha, n - 1}} t_{n - 1} (x) dx = cf
\end{equation}
We make use of the symmetry of this distribution and chose 
$$
\alpha = \frac{1 - cf}{2}
$$
so that
\begin{equation}
\int_{-\infty}^{t_{\alpha, n - 1}} t_{n - 1} (x) dx = \alpha
\end{equation}
and we will use the inverse survival function of ```scipy.stat.t``` .

In [ ]:
confidence_level

In [ ]:
t_CL_N = scipy.stats.t.isf((1-confidence_level)/2, sample_size - 1)

In [ ]:
t_CL_N

Note that for sufficiently large $n$ the $t$ distribution is close to a Gaussian and for a confidence level of 68\% this number is close to 1. We will not discuss the $t$ distribution further.

In [ ]:
sample_sd = np.std(samples, axis = (1), ddof = 1) ## ddof = 1 for Bessel correction

In [ ]:
mean_sd = t_CL_N * sample_sd/sqrt(sample_size)

In [ ]:
mean_sd

In [ ]:
plt.hist(mean_sd)

In [ ]:
sample_devs = (sample_means - mu)/mean_sd

In [ ]:
sample_devs

In [ ]:
plt.hist(sample_devs)

#plt.yscale('log')
plt.xlabel('$(x - \\overline{x}) / \\sigma$')


plt.savefig('Deviation_MultipleObservation.svg')

In [ ]:
np.sum(np.abs(sample_devs) < 1)/len(sample_devs)

In [ ]:
np.sum(np.abs(sample_devs) < 3)/len(sample_devs)

This is very close to what we have expected.

In [ ]:
plt.hist(mean_sd)

Our typical uncertainty from this method is the following. We will keep this and see later if we can do better.

In [ ]:
uncertainty_from_mean = np.mean(mean_sd)

In [ ]:
uncertainty_from_mean

## Poisson statistics

In [ ]:
mu

We have seen earlier that the distribution of counts in the tiles is not symmetric, has a mean of $\mu$ and a standard deviation of $\sqrt{\mu}$. This is the Poisson distribution.

The probability to count $n$ events when the mean (the expectation) is $\mu$ is given by :
\begin{equation}
\text{prob}(n|\mu) = 
\frac{\mu^n}
{n!}
e^{-\mu}
\end{equation}


In [ ]:
plt.stairs(counts, np.append(values, values[-1]+1) - 0.5,
           fill=True,
           label = 'observations'
          )

plt.plot(values,
         scipy.stats.poisson.pmf(values, mu) *np.sum(counts),
         marker = 'o',
         label = 'Poisson ($\\mu = 10$)'
        )

plt.legend()

plt.savefig('Poisson.svg')

What is the probability of measuring exactly mu?

In [ ]:
scipy.stats.poisson.pmf(mu, mu)

while we have

In [ ]:
f'{counts[values == mu]} / {np.sum(counts)} = {counts[values == mu]/np.sum(counts)}'

In [ ]:
scipy.stats.poisson.pmf(100,100)*100

What is the probability of having a completely empty tile?

In [ ]:
scipy.stats.poisson.pmf(0, mu)*100

In [ ]:
f'{counts[values == 0]} / {np.sum(counts):1.0e} = {counts[values == 0]/np.sum(counts)*100}%'

In [ ]:
scipy.stats.poisson.pmf(0, 100)

What is the probability of measuring less than or equal the mean?

In [ ]:
scipy.stats.poisson.cdf(mu, mu)

In [ ]:
f'{np.sum(counts[values <= 10])} / {np.sum(counts):1.0e} = {np.sum(counts[values <= 10])/np.sum(counts)*100}%'

What is the probability of measuring more than $\mu$ ?

In [ ]:
scipy.stats.poisson.sf(mu, mu)

In [ ]:
f'{np.sum(counts[values > 10])} / {np.sum(counts):1.0e} = {np.sum(counts[values > 10])/np.sum(counts)*100}%'

What is the probability of measuring higher than our maximum value?

In [ ]:
np.max(values)

In [ ]:
scipy.stats.poisson.sf(np.max(values), mu) * 100

## Likelihood Profile

We have ten consecutive measurements, e.g.:

In [ ]:
samples[0]

We know the probability of the counts in each individual tile for a given rate. The overall probability is the product of that:
\begin{equation}
\mathcal{L} = \text{prob} = \prod_{i = 1}^{i=n} \text{prob}(x_i|\mu)
\end{equation}

Let's calculate the probability for all of our samples:

In [ ]:
(np.prod(scipy.stats.poisson.pmf(samples, mu), axis = (1)))

In [ ]:
plt.hist(np.log10(np.prod(scipy.stats.poisson.pmf(samples, mu), axis = (1))), log = True)

plt.xlabel('$\\log_{10} \\mathcal{L}$')

plt.savefig('Ldistribution.svg')

Now let's scan the probability for different values of $\mu$ around the true value:

In [ ]:
mus = np.arange(mu - 3, mu + 3, step = 0.1)

In [ ]:
x = np.prod(np.array(list(map(lambda mu : scipy.stats.poisson.pmf(samples[0], mu), mus))), axis = (1))

In [ ]:
x.shape

In [ ]:
x = np.array(list(map(
    lambda sample : np.prod(np.array(list(map(lambda mu : scipy.stats.poisson.pmf(sample, mu), mus))), axis = (1)), 
    samples[:])))

In [ ]:
x.shape

In [ ]:
plt.plot(mus, x[0])

plt.xlabel('$\\mu$')
plt.ylabel('$\\mathcal{L}$')

plt.savefig('Lprofile.svg')

In [ ]:
plt.plot(mus, x[18])

This function has a very clear maximum, which we can use to estimate the true value.

In [ ]:
ret = plt.plot(mus, np.transpose(x[:10]))

#plt.yscale('log')

plt.xlabel('$\\mu$')
plt.ylabel('$\\mathcal{L}$')

plt.savefig('Lprofile2.svg')

Finding the maximum of this profile will give us an estimate of the rate from the sample. This would need the first derivative. Using the second derivative we could even obtain information on the uncertainty. 

### Log Likelihood
As we are looking for the maximum of the likelihood distribution and the logarithm is strictly increasing, we can also work with the logarithm of the likelihood. This will also simplify certain calculations.

In particular the product of the probabilities transforms into a sum:
$$
\mathcal{l}  = \ln\mathcal{L} = \ln \left( \prod_{i = 1}^{i=n} \text{prob}(x_i|\mu) \right) =   \sum_{i = 1}^{i=n} \ln \left( \text{prob}(x_i|\mu)  \right)
$$


In [ ]:
ls = np.array(list(map(
    lambda sample : np.sum(np.array(list(map(lambda mu : np.log(scipy.stats.poisson.pmf(sample, mu)), mus))), axis = (1)), 
    samples[:])))

In [ ]:
ls.shape

In [ ]:
plt.plot(mus, ls[0])

plt.xlabel('$\\mu$')
plt.ylabel('$\\ln \\mathcal{L}$')

plt.savefig('lnLprofile.svg')

For the Poisson distribution we can simplify even further:
\begin{align*}
\mathcal{l} & = \ln\mathcal{L}
= \sum_{i = 1}^{i=N} \ln \left( \frac{\mu^n_i}
{n_i!}
e^{-\mu}  \right)
\\
{} &= \sum_{i = 1}^{i=N}
\left(
- \mu + n_i \ln{\left(\mu \right)} - \ln{\left(n_i! \right)}
  \right)
\\
{} &= - N \mu  + \ln{\left(\mu \right)} \sum_{i = 1}^{i=N}
   n_i  -
  \sum_{i = 1}^{i=N} \ln{\left(n_i! \right)}
\end{align*}

The last term is a constant and does depend only on our data set, and not on $\mu$. If we want to study the likelihood profile in dependance of $\mu$ we can ignore this term and define a test statistic as
$$
TS = - N \mu  + \ln{\left(\mu \right)} \sum_{i = 1}^{i=N}
   n_i
$$
Which is close to the Cash statistic.

If we want to find the maximum of this distribution we need to calculate the value $\mu$ for which the deriviative vanishes. The derivative of $\ln{\mathcal{l}}$ is
$$
\frac{d~\ln{\mathcal{L}}}{d\mu} = - N + \frac{\sum_{i=1}^{N} {n}_{i}}{\mu}
$$

so that 
$$
\mu = \frac{\sum_{i=1}^{N} {n}_{i}}{N}
$$
which is simply the mean.

While this looks like a fancy way to calculate the mean the likelihood profile is much more powerful. In our example the the expectation value in each tile is exactly the same. This is not always the case. Imagine tiles of different sizes, or tiles being measurements with different durations. You can still calculate the probability for each tile and get the likelihood profile.

The likelihood profile may even depend on several parameters, and an analytical solution or derivitative might no exist. In this case the negative of our test statistics, $-ts$, can be numerically minimised.

In [ ]:
def ts (mu) :
    return - (np.log(mu)*np.sum(counts_in_tiles) -tiles*mu)

In [ ]:
optres = scipy.optimize.minimize(ts, [1],
                                 bounds=[(0,np.inf)], 
                                 hess='2-point'
                                )

In [ ]:
optres

In [ ]:
optres.x

In [ ]:
sample_mus = (np.sum(samples, axis = 1))/samples.shape[1]

In [ ]:
plt.hist(sample_mus, bins = np.arange(int(np.min(sample_mus)), int(np.max(sample_mus))+2) -0.5)

### Estimating Uncertainty

#### Hessian matrix

The negative of the inverse of the Hessian matrix of the logarithm of the likelihood is an estimate for the covariance of the estimated parameters. (This is another reason to use the log of the likelihood.) Certain conditions have to be met, in particular the likelihood has to be normally distributed around the maximum.

In our example we have only one single parameter and the Hessian is simply the second derivative of $\ln{\mathcal{L}}$ which is
$$
h = \frac{d^2~\ln{\mathcal{L}}}{d\mu^2} = - \frac{\sum_{i=1}^{N} {n}_{i}}{\mu^{2}}
$$

So the standard deviation of $\mu$ is
$$
\sigma_\mu = \frac{1}{\sqrt{-h}} = \frac{\mu}{\sqrt{\sum_{i=1}^{N} {n}_{i}}}
$$
It is the mean, as expected from a Poisson distribution, divided by the total number of counts. There more events, the better the estimate!

It can also be written as
$$
\sigma_\mu = \frac{\sqrt{\sum_{i=1}^{N} {n}_{i}}}{N}
$$

In the case that the derivative cannot be obtained analytically the inverse Hessian can be obtained from a numerical minimisation.

In [ ]:
np.sqrt(np.sum(counts_in_tiles))/tiles

In [ ]:
optres.hess_inv.todense()

In [ ]:
sqrt(optres.hess_inv.todense())

##### Test

In [ ]:
mu_sd_hessian = np.sqrt(np.sum(samples, axis = 1))/samples.shape[1]

In [ ]:
plt.hist(mu_sd_hessian)

plt.yscale('log')



This looks very good. The spread is much lower then what we have found before.

In [ ]:
sample_devs_hessian = (sample_mus - mu)/mu_sd_hessian

In [ ]:
sample_devs_hessian

In [ ]:
plt.hist(sample_devs_hessian)

plt.yscale('log')

plt.xlabel('$(x - \\overline{x}) / \\sigma$')

plt.savefig('Deviation_Hessian.svg')

In [ ]:
np.sum(np.abs(sample_devs_hessian) < 1)/len(sample_devs_hessian)

In [ ]:
np.sum(np.abs(sample_devs_hessian) < 3)/len(sample_devs_hessian)

This is very much what we have expected.

In [ ]:
plt.hist(mu_sd_hessian)

Our typical uncertainty from this method is the following. We will keep this and see later if we can do better.

In [ ]:
uncertainty_from_hessian = np.mean(mu_sd_hessian)

In [ ]:
uncertainty_from_hessian

We have some improvement from our mean method.

#### Using Wilks' theorem

So far we have used the first and second derivatives of $\ln{\mathcal{L}}$ or at least assumed that the profile is normal distributed around the maximum. We now use a different approach which would also work for other profiles.

Wilks' theorem states that the ratio of the likelihoods of two models $0$ and $1$
$$
D = -2 \ln{\frac{\mathcal{L_0}}{\mathcal{L_1}}}
$$
approaches asymptotically when the sample sizes goes to infinity a $\chi^2$ distribution and will give the probabilty of model 1 being a statistical fluctuation of model 0, the null hypothesis. 
This can be written as
$$
D = -2 ( \ln {\mathcal{L_0}} - \ln {\mathcal{L_1}}) = 2 ( \ln {\mathcal{L_1}} - \ln {\mathcal{L_0}})
$$
which is another reason why we use the logarithm of the likelihood and explains the factor 2 in the Cash statistics.
The models have to be nested models: Model 1 is the alternative model, and model 0 is a special case of the alternative model, i.e. it is Model 1 with a subset of the parameters frozen. The degrees of freedom is then the difference of the number of free parameters in the model.

In order to construct a confidence level we will scan the parameter space (the allowed values for $\mu$) and calculate how close they are to the best fit value $\mu_0$, obtained from the maximum of the likelihood distribution. The null hypothesis is that $\mu$ is fixed to the value under investigation and all other parameters (in the case of a multiparameter model) are optimised (likelihood maximised). The alternative hypothesis is with all parameters free and optimised, i.e. we compare with the best-fit parameter $\mu_0$.

This way we can construct a test statistic profile, $D(\mu)$. Obviousily, $D(\mu_0) = 0$. Further on, as the likelihood is maximal at $\mu_0$, $D$ positive for all other values of $\mu$.

Let's take a look at it for a Poisson distribution:

$$
D = 2 \left(N \mu - \ln{\left(\mu \right)} \sum_{i=1}^{N} {n}_{i} + \sum_{i=1}^{N} \ln{\left({n}_{i}! \right)}\right) + 2 \left(- N \mu_{0} + \ln{\left(\mu_{0} \right)} \sum_{i=1}^{N} {n}_{i} - \sum_{i=1}^{N} \ln{\left({n}_{i}! \right)}\right)
$$

We see that the term $\sum_{i=1}^{N} \ln{\left({n}_{i}! \right)}$ cancels out. This is the reason for ignoring it in the Cash statistics.

\begin{align*}
D  &= 2 N \mu - 2 N \mu_{0} + 2 \left(- \ln{\left(\mu \right)} + \ln{\left(\mu_{0} \right)}\right) \sum_{i=1}^{N} {n}_{i} \\
{} &= 2 N \mu - 2 N \mu_{0} + 2 n \left(- \ln{\left(\mu \right)} + \ln{\left(\mu_{0} \right)}\right)
\\
{} & = 2 N \mu + 2 n \left(- \ln{\left(\mu \right)} + \ln{\left(\frac{n}{N} \right)}\right) - 2 n
\end{align*}

The number of degrees of freedom in this case is one.


In [ ]:
def D(mu, N, n) :
    """N measurements, sum of counts = n"""
    
    return (2*N*mu + 2*n*(-np.log(mu) + np.log(n/N)) - 2*n)

In [ ]:
mus

In [ ]:
D(mus, samples.shape[1], np.sum(samples[0]))

In [ ]:
Ds = list(map(lambda sum : D(mus, samples.shape[1], sum), np.sum(samples, axis = 1)))

In [ ]:
plt.plot(mus, Ds[0])
#plt.plot(mus, Ds[1])

plt.ylim(0)

plt.ylabel('$D$')

plt.xlabel('$\\mu$')

plt.savefig('Dprofile.svg')

In [ ]:
scipy.stats.chi2.ppf(0.68, df = 1)

In [ ]:
plt.plot(mus, Ds[0])
#plt.plot(mus, Ds[1])

plt.ylim(0)

plt.ylabel('$D$')

plt.xlabel('$\\mu$')

secax_y = plt.gca().secondary_yaxis('right', 
                             functions=(lambda D : scipy.stats.chi2.cdf(D, df = 1), 
                                        lambda p : scipy.stats.chi2.ppf(p, df = 1)
                                        )
                            )
secax_y.set_ylabel('$\\chi^2$ probability')


#plt.savefig('Dprofile.svg')

In [ ]:
plt.plot(mus,
         scipy.stats.chi2.cdf(Ds[0], df = 1)
        )

plt.ylabel('$\\chi^2$ probability')

plt.xlabel('$\\mu$')

plt.ylim(0,0.9999)


secax_y = plt.gca().secondary_yaxis('right', 
                             functions=(lambda p : scipy.stats.chi2.ppf(p, df = 1),
                                        lambda D : scipy.stats.chi2.cdf(D, df = 1)
                                        )
                            )
secax_y.set_ylabel('$D$')

secax_y.set_yticks([0,1,2,3,4,5])

plt.savefig('Chi2Probability.svg')

In [ ]:
scipy.stats.chi2.cdf([0, 1, 2, 3, 4], df = 1)

In [ ]:
scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf([1, 2, 3]), df = 1)

In [ ]:
1 - 2*scipy.stats.norm.sf([1, 2, 3])

The individual values follow a $\chi^2$ distribution with 1 degree of freedom. 68.27% of the values in a $\chi^2$ distribution are lower than 1:

In [ ]:
scipy.stats.chi2.ppf(confidence_level,1)

So the limits of our confidence interval are where $D$ reaches 1.

In [ ]:
scipy.optimize.fsolve(lambda mu : D(mu, sample_size, np.sum(samples[0])) - 1, np.mean(samples[0])*np.array([0.9, 1.1]))

In [ ]:
conf_interval = np.array(list(
    map(lambda x : scipy.optimize.fsolve(lambda mu : D(mu, sample_size, np.sum(x)) - 1, np.mean(x)*np.array([0.9, 1.1])),
                                         samples)))

In [ ]:
conf_interval_3s = np.array(list(
    map(lambda x : scipy.optimize.fsolve(lambda mu : D(mu, sample_size, np.sum(x)) - 9, np.mean(x)*np.array([0.9, 1.1])),
                                         samples)))

In [ ]:
conf_interval

In [ ]:
sample_sd

In [ ]:
sample_mus

In [ ]:
mus

In [ ]:
index = 0

plt.plot(mus, Ds[index])

maxD = np.nanmax(Ds[index])

plt.ylim(0)

plt.ylabel('$D$')

plt.xlabel('$\\mu$')

plt.vlines(sample_mus[index], 0, maxD,
           ls = 'dotted', color = 'green',
           label = 'mean'
          )

plt.vlines(sample_mus[index] + np.array([mu_sd_hessian[index], -mu_sd_hessian[index]]), 0, maxD,
           ls = 'dotted', color = 'red',
           label = 'Hessian uncertainty'
          )

plt.vlines(conf_interval[index], 0, maxD,
           ls = 'dotted', color = 'darkred',
           label = "Wilk's uncertainty"
          )


plt.legend()

plt.xlim(sample_mus[index] + 2*np.array([-mu_sd_hessian[index], +mu_sd_hessian[index]]))

plt.savefig('Dprofile_withUnc.svg')

In [ ]:
conf_interval < mu

In [ ]:
np.sum(np.array(list(map(lambda x : np.logical_and(x[0]<= mu, x[1] >= mu) , conf_interval))))

In [ ]:
np.sum(np.array(list(map(lambda x : np.logical_and(x[0]<= mu, x[1] >= mu) , conf_interval_3s))))

## Example

In [ ]:
len(counts_in_tiles) % 6

In [ ]:
samples2 = np.reshape(counts_in_tiles[:-(len(counts_in_tiles) % 6)], shape = (-1, 6))

In [ ]:
samples2

In [ ]:
samples2[0]

In [ ]:
samples2[0][0]

In [ ]:
np.sum(samples2[0][1:3])

In [ ]:
np.sum(
samples2[0][3:]
)

In [ ]:
exsamples = np.array([samples2[:,0], np.sum(samples2[:,1:3], axis = 1), np.sum(samples2[:,3:], axis = 1)]).transpose()

In [ ]:
exsamples

In [ ]:
np.sum(exsamples[1:4,2])

### $A_x$ par marginalisation

In [ ]:
def Check_Ax(counts, A_x, sigma = 1) :

    n_1 = counts[0]
    n_2 = counts[1]
    n_x = counts[2]

    Ax_best = (3*n_x/(n_1 + n_2))

    Ax_std = (3*sqrt(n_x)*sqrt(-1/(-n_1 - n_2)**3)*sqrt(n_1 + n_2 + n_x))

    return abs(Ax_best-A_x) < sigma*Ax_std


In [ ]:
np.sum(list(map( lambda counts : Check_Ax (counts, A_x = 3), exsamples))) / exsamples.shape[0]

In [ ]:
np.sum(list(map( lambda counts : Check_Ax (counts, A_x = 3, sigma = 3), exsamples))) / exsamples.shape[0]

### $\mu$ par marginalisation

In [ ]:
def Check_mu(counts, mu, sigma = 1) :

    n_1 = counts[0]
    n_2 = counts[1]
    n_x = counts[2]

    mu_best = ((1/3)*n_1 + (1/3)*n_2)

    mu_std = ((1/3)*sqrt(n_1 + n_2))

    return abs(mu_best-mu) < sigma*mu_std


In [ ]:
np.sum(list(map( lambda counts : Check_mu (counts, mu = 10), exsamples))) / exsamples.shape[0]

In [ ]:
np.sum(list(map( lambda counts : Check_mu (counts, mu = 10, sigma = 3), exsamples))) / exsamples.shape[0]

### 2d Likelihood Profile

In [ ]:
def DeltaTS (counts, mu, A_x) :

    n_1 = counts[0]
    n_2 = counts[1]
    n_x = counts[2]
    
    return (2*A_x*mu + 6*mu - 2*n_1*math.log(mu) + 2*n_1*math.log(n_1 + n_2) - 2*n_1*math.log(3) - 2*n_1 - 2*n_2*math.log(mu) + 2*n_2*math.log(n_1 + n_2) - 2*n_2*math.log(3) - 2*n_2 - 2*n_x*math.log(A_x) - 2*n_x*math.log(mu) + 2*n_x*math.log(n_x) - 2*n_x)

In [ ]:
DeltaTS (exsamples[0], mu = 10, A_x = 3)

In [ ]:
TS = np.array(list(map( lambda counts : DeltaTS (counts, mu = 10, A_x = 3), exsamples)))

In [ ]:
TS

In [ ]:
np.sum(TS < scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf(1), df = 2)) / exsamples.shape[0]

In [ ]:
np.sum(TS < scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf(3), df = 2)) / exsamples.shape[0]